# Hat game demo
In this notebook we
1. download sample corpus
1. train few models on it
1. write a class which will follow AbstractPlayer conventions
1. finally, play the game between local models (players) and one remote :)

Note that exact output may be not reproducible as the remote player could change over time or even fail/timeout at some point

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

from data.utils import merge_all_files
from the_hat_game.game import Game
from the_hat_game.players import PlayerDefinition, AbstractPlayer, RemotePlayer

pd.set_option('display.max_colwidth', 200)

/Users/aguschin/.local/share/virtualenvs/the-hat-game-gitlab-5E7X0_QA/lib/python3.8/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## download sample text corpus

In [2]:
%%bash
cd texts
wget --quiet http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
tar -zxf 20news-19997.tar.gz
rm 20news-19997.tar.gz

In [3]:
file_path =  Path('texts/20-newsgroups.txt')
folder = Path('texts/20_newsgroups/')

In [4]:
with open(file_path, 'w', encoding='utf-8') as f_write:
    files = list(folder.rglob('*'))
    for object_path in tqdm(files):
        if object_path.is_dir():
            continue
        with open(object_path, encoding='latin-1') as stream:
            for line in stream:
                f_write.write(line)

100%|███████████████████████████████████| 20017/20017 [00:07<00:00, 2650.21it/s]


In [5]:
!wc -w {file_path}

 6046669 texts/20-newsgroups.txt


In [6]:
!head -5 {file_path}

Newsgroups: talk.politics.mideast
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!uunet!brunix!doorknob!hm
From: hm@cs.brown.edu (Harry Mamaysky)
Subject: Heil Hernlem 
In-Reply-To: hernlem@chess.ncsu.edu's message of Wed, 14 Apr 1993 12:58:13 GMT


## Train several models

In [7]:
%%time

model_skipgram = fasttext.train_unsupervised(str(file_path), model='skipgram', dim=5)
model_cbow = fasttext.train_unsupervised(str(file_path), model='cbow', dim=16)
model_skipgram2 = fasttext.train_unsupervised(str(file_path), model='skipgram', dim=10)

CPU times: user 6min 16s, sys: 2.75 s, total: 6min 19s
Wall time: 39.9 s


In [8]:
model_skipgram.words[1]

'the'

In [9]:
len(model_skipgram.words)

72228

In [10]:
model_skipgram['song']

array([-0.06952401,  0.34956253, -1.7137849 , -1.0474566 ,  0.06292222],
      dtype=float32)

In [11]:
!mkdir models
model_skipgram.save_model('models/skipgram.model')
model_skipgram2.save_model('models/skipgram2.model')
model_cbow.save_model('models/cbow.model')

mkdir: models: File exists


In [12]:
!ls -lh models

total 579968
-rw-r--r--  1 aguschin  staff   132M May 27 12:21 cbow.model
-rw-r--r--  1 aguschin  staff    42M May 27 12:21 skipgram.model
-rw-r--r--  1 aguschin  staff    83M May 27 12:21 skipgram2.model


## Players' classes for fasttext models

In [13]:
class LocalFasttextPlayer(AbstractPlayer):
    def __init__(self, model):
        self.model = model

    def find_words_for_sentence(self, sentence, n_closest):
        neighbours = self.model.get_nearest_neighbors(sentence)
        words = [word for similariry, word in neighbours][:n_closest]
        return words

    def explain(self, word, n_words):
        return self.find_words_for_sentence(word, n_words)

    def guess(self, words, n_words):
        return self.find_words_for_sentence(' '.join(words), n_words)

In [14]:
# check remotely deployed service
remote_player = RemotePlayer('https://obscure-everglades-02893.herokuapp.com')

print(remote_player.explain('work', 10))
print(remote_player.guess(['job', 'employee', 'office'], 5))

['work', 'discontent', 'probably:', 'lopid', 'gives', 'putty', 'refund', 'strangest', 'enuff', 'inovative']
['bars;', 'earnings', 'appellate', 'discoverd', 'phage']


In [15]:
local_player = LocalFasttextPlayer(model_skipgram)
print(local_player.explain('work', 10))
print(local_player.guess(['job', 'employee', 'office'], 5))

['interstate', 'robust,', 'shadow', 'one),', 'removes', 'dime', "driver's", 'fills.', 'drawing.', 'mice.']
['forefront', '[some', 'acoustic', 'concertina', 'press.']


## Playing game!

In [16]:
N_EXPLAIN_WORDS = 10
N_GUESSING_WORDS = 5
N_ROUNDS = 1
CRITERIA = 'soft'

PLAYERS = [
    PlayerDefinition('HerokuOrg team', RemotePlayer('https://obscure-everglades-02893.herokuapp.com')),
    PlayerDefinition('skipgram team', LocalFasttextPlayer(model_skipgram)),
    PlayerDefinition('skipgram2 team', LocalFasttextPlayer(model_skipgram2)),
    PlayerDefinition('cbow team', LocalFasttextPlayer(model_cbow))
]

WORDS = ['dollar', 'percent', 'billion', 'money']

game = Game(PLAYERS, WORDS, CRITERIA, N_ROUNDS, N_EXPLAIN_WORDS, N_GUESSING_WORDS, random_state=0)
game.run(verbose='print_logs', complete=False)

HOST to EXPLAINING PLAYER (HerokuOrg team): the word is "billion"
EXPLAINING PLAYER (HerokuOrg team) to HOST: my wordlist is ['billion', 'dings', '>near', 'pellets', 'prelude', '100th', 'ammo', 'calibre', 'mile', 'corollas']
HOST TO EXPLAINING PLAYER (HerokuOrg team): cleaning your word list. Now the list is ['dings', 'near', 'pellets', 'prelude', 'th', 'ammo', 'calibre', 'mile', 'corollas']

===ROUND 1===

HOST: ['dings']
GUESSING PLAYER (skipgram team) to HOST: ['(long', 'orbiter', 'Pluto', 'surges', 'Titan']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['headings', 'windings', 'marks', 'joints', 'standings']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['earnings', 'headings', 'copayments', 'segments', '>parts']
HOST: False

===ROUND 2===

HOST: ['dings', 'near']
GUESSING PLAYER (skipgram team) to HOST: ['carbon,', 'Torx', "Mercury's", 'Upon', 'Driving']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['landings,', 'plotted', 'windings', 'ounces', 'close-up']
HOST

,"Explanation for ""billion"" (HerokuOrg team)",Guess (skipgram team),Guess (skipgram2 team),Guess (cbow team)
0,[dings],"[(long, orbiter, Pluto, surges, Titan]","[headings, windings, marks, joints, standings]","[earnings, headings, copayments, segments, >parts]"
1,"[dings, near]","[carbon,, Torx, Mercury's, Upon, Driving]","[landings,, plotted, windings, ounces, close-up]","[breakfasts, fielding, machbanding, colliding, centering]"
2,"[dings, near, pellets]","[80%, tours, navy, rosters, drills]","[""tail, ceiling, turnaround, shrine, 'center]","[putouts, arrives, hotels, fronts, stacks]"
3,"[dings, near, pellets, prelude]","[chapter., lawnmower, >4., pulsars,, 12-story]","['center, remainder,, ""C"", ""tail, ""SCHWARZENEGGER""]","[collect, snide, cyanide, handheld, spy]"
4,"[dings, near, pellets, prelude, th]","[Thing,, leader????, pulsars,, DJ>, ""Power]","['center, ""SCHWARZENEGGER"", >center, F-holder, remainder,]","[tackle, spy, cyanide, mass-market, sunlight]"
5,"[dings, near, pellets, prelude, th, ammo]","[summer?, Thing,, spectacular,, pulsars,, tradeoff]","['center, greenhouse, ""tail, ""SCHWARZENEGGER"", remainder,]","[cyanide, sunlight, control..., tackle, brownbladerunnersugarcubeselectronicblaylockpowersspikeleekatebushhamcornpizza]"
6,"[dings, near, pellets, prelude, th, ammo, calibre]","[110V,, >both, (3),, tuberculosis, brochure]","[blackout, greenhouse, flashers., remainder,, ""C""]","[control..., mass-market, >risk, blasphemous, control:]"
7,"[dings, near, pellets, prelude, th, ammo, calibre, mile]","[(3),, 110V,, tuberculosis, |is, brochure]","[greenhouse, blackout, flashers., remainder,, ""C""]","[mass-market, seminar, inhibitor, target), whitespace]"
8,"[dings, near, pellets, prelude, th, ammo, calibre, mile, corollas]","[""SCHWARZENEGGER"", Mars?, :a, (3),, Cavalier,]","[bingo, greenhouse, flashers., fibreglass, Zodiacal]","[brownbladerunnersugarcubeselectronicblaylockpowersspikeleekatebushhamcornpizza, mass-market, cyanide, transformer, karma]"


HOST to EXPLAINING PLAYER (skipgram team): the word is "money"
EXPLAINING PLAYER (skipgram team) to HOST: my wordlist is ['cleanest', 'closely.', 'frustrating', 'diet,', 'constantly.', 'visible,', 'aiming', 'counter.', 'noise.', 'chant']
HOST TO EXPLAINING PLAYER (skipgram team): cleaning your word list. Now the list is ['cleanest', 'closely', 'frustrating', 'diet', 'constantly', 'visible', 'aiming', 'counter', 'noise', 'chant']

===ROUND 1===

HOST: ['cleanest']
GUESSING PLAYER (HerokuOrg team) to HOST: ['cleanest', 'easing', 'blocking', 'flattery', 'flattering']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['cleanly', 'constantly.', 'x-rays', 'unused', 'constantly,']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['weakest', 'cleanse', 'closest', 'template', 'seal']
HOST: False

===ROUND 2===

HOST: ['cleanest', 'closely']
GUESSING PLAYER (HerokuOrg team) to HOST: ['>cases', 'disposable', 'reconditioned', 'damaging', 'visions']
HOST: False
GUESSING PLAYER (skipgram2 team) 

,"Explanation for ""money"" (skipgram team)",Guess (HerokuOrg team),Guess (skipgram2 team),Guess (cbow team)
0,[cleanest],"[cleanest, easing, blocking, flattery, flattering]","[cleanly, constantly., x-rays, unused, constantly,]","[weakest, cleanse, closest, template, seal]"
1,"[cleanest, closely]","[>cases, disposable, reconditioned, damaging, visions]","[scenarios, apparently, >>also, liscenced, successful,]","[invisible, desperately, closely, visible, hideous]"
2,"[cleanest, closely, frustrating]","[prongs, >>>the, parting, arcing, >figures]","[eliminated., goodies, memorize, discover, most.]","[frustrating, abroad, dismantling, exaggerating, concentrating]"
3,"[cleanest, closely, frustrating, diet]","[visions, floggings, smuggled, safe, legalizing]","[2%., spectacle, goodies, eliminated., many).]","[abroad, passive-aggressive, eliminate, brightest, cleansing""]"
4,"[cleanest, closely, frustrating, diet, constantly]","[>>>two, arcing, floggings, smuggled, remedy]","[goodies, x-rays, one-half, many)., successful,]","[abroad, torture, desperately, incontrovertibly, prolong]"
5,"[cleanest, closely, frustrating, diet, constantly, visible]","[smuggled, safe, legalizing, arcing, trapping]","[goodies, x-rays, 2%., one-half, successful,]","[passive-aggressive, intrusive, incontrovertibly, invisible, abroad]"
6,"[cleanest, closely, frustrating, diet, constantly, visible, aiming]","[comb, >>>two, smuggled, arcing, >pay]","[x-rays, goodies, >(which, >>>a, 2%.]","[dismantling, concentrating, cleansing"", abroad, exaggerating]"
7,"[cleanest, closely, frustrating, diet, constantly, visible, aiming, counter]","[parting, arcing, comb, >>>two, than]","[one-half, goodies, x-rays, JFK, EKG]","[abroad, torture, hand-held, seminar, passive-aggressive]"
8,"[cleanest, closely, frustrating, diet, constantly, visible, aiming, counter, noise]","[parting, legit, spit, prongs, >figures]","[one-half, JFK, x-rays, CJF>, goodies]","[abroad, torture, passive-aggressive, furniture, seminar]"
9,"[cleanest, closely, frustrating, diet, constantly, visible, aiming, counter, noise, chant]","[comb, >pay, outweighs, >>>two, pains]","[one-half, JFK, CJF>, many)., goodies]","[abroad, torture, passive-aggressive, seminar, surpass]"


HOST to EXPLAINING PLAYER (skipgram2 team): the word is "percent"
EXPLAINING PLAYER (skipgram2 team) to HOST: my wordlist is ['percent.', 'percent,', 'civic', 'mixture', '7.3', 'mixture,', '20-39', '7.3.', 'temper', '42%']
HOST TO EXPLAINING PLAYER (skipgram2 team): cleaning your word list. Now the list is ['civic', 'mixture', 'temper']

===ROUND 1===

HOST: ['civic']
GUESSING PLAYER (HerokuOrg team) to HOST: ['civic', 'o-rings', '2-3', 'hotdogs', 'breeze']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['pregnancy.', 'accidental,', 'hospital.', 'froze', 'bleachers']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['pelvic', '<of', 'perpetual', 'civil', 'civility']
HOST: False

===ROUND 2===

HOST: ['civic', 'mixture']
GUESSING PLAYER (HerokuOrg team) to HOST: ['chassis', 'yielding', 'loaner', 'averaging', 'light"']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['cocktail', '(carrying', 'Oilers.', 'gravel', 'parachute']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['h

,"Explanation for ""percent"" (skipgram2 team)",Guess (HerokuOrg team),Guess (skipgram team),Guess (cbow team)
0,[civic],"[civic, o-rings, 2-3, hotdogs, breeze]","[pregnancy., accidental,, hospital., froze, bleachers]","[pelvic, <of, perpetual, civil, civility]"
1,"[civic, mixture]","[chassis, yielding, loaner, averaging, light""]","[cocktail, (carrying, Oilers., gravel, parachute]","[homicides/population, population!, population, chromaticity, civility]"
2,"[civic, mixture, temper]","[subway, $2-billion, 80%, interstates, >approximately]","[deadline, 15-day, Goering, attendance, circles]","[polynomial, symposium, titanium, temperature, fatal]"


HOST to EXPLAINING PLAYER (cbow team): the word is "dollar"
EXPLAINING PLAYER (cbow team) to HOST: my wordlist is ['dollar/pound', 'pillion', 'trillion', '$1billion', 'bills', 'zillion', 'billion', 'dollar.', 'dollars).', '>million']
HOST TO EXPLAINING PLAYER (cbow team): cleaning your word list. Now the list is ['pillion', 'trillion', 'billion', 'bills', 'zillion', 'million']

===ROUND 1===

HOST: ['pillion']
GUESSING PLAYER (HerokuOrg team) to HOST: ['pillion', 'crawl', 'ouch', 'gloves', 'players:']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['slid', 'lock,', 'inch.', 'wheel,', 'bikes']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['deck,', 'nd', '(how', 'pockets', 'ticks']
HOST: False

===ROUND 2===

HOST: ['pillion', 'trillion']
GUESSING PLAYER (HerokuOrg team) to HOST: ['showroom', 'saddle', 'chatter', 'headline', 'spot']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['nanosecond,', '4-door', 'scoreboard', 'Too', 'strobe']
HOST: False
GUESSING PLAYER (ski

,"Explanation for ""dollar"" (cbow team)",Guess (HerokuOrg team),Guess (skipgram team),Guess (skipgram2 team)
0,[pillion],"[pillion, crawl, ouch, gloves, players:]","[slid, lock,, inch., wheel,, bikes]","[deck,, nd, (how, pockets, ticks]"
1,"[pillion, trillion]","[showroom, saddle, chatter, headline, spot]","[nanosecond,, 4-door, scoreboard, Too, strobe]","[pigs, all-round, towers, 20-30, >both]"
2,"[pillion, trillion, billion]","[ranged, lighters, dining, cruiser, seating]","[finals,, bolts, walkman, Saab, Saturn.]","[billion., $1billion, zillion, pigs, all-round]"
3,"[pillion, trillion, billion, bills]","[boiler, taxi, bracelet, road;, overhauled]","[line:, 2-door, sedan,, gimick,, turnaround]","[all-round, >both, goons, breakaway, With]"
4,"[pillion, trillion, billion, bills, zillion]","[opening, bracelet, gearing, fire-breathing, folds]","[2-door, eg, Thing., line:, Saab]","[billion., >both, $1billion, zillion, all-round]"
5,"[pillion, trillion, billion, bills, zillion, million]","[bracelet, paced, opening, casings, week]","[guitar, Thing., Turns, line:, turnaround]","[zillion, 700.000, breakaway, billion., $1billion]"


## View final game report

In [17]:
game.report_results(each_game=True)

=== Team scores in each game ===


,HerokuOrg team,skipgram team,skipgram2 team,cbow team
game,,,,
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0


=== Team scores, summary ===


,explaining,guessing,total
HerokuOrg team,0,0,0.0
cbow team,0,0,0.0
skipgram team,0,0,0.0
skipgram2 team,0,0,0.0
